> ## Python実践データ分析100本ノック解説（ノック11〜ノック20)

### Pytho実践データ分析100本ノックのノック11からノック20の解答とコードです。参考になりますと幸いです。

* #### データの加工について学ぶ
* #### エクセルで人間が打ち込んだ、「汚い」データを題材にする

> ## ノック11 : データの読み込み

In [1]:
import pandas as pd


# 売り上げデータの読み込み
uriage_data = pd.read_csv('../input/bookfile2/uriage.csv')
uriage_data.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美
1,2019-07-13 13:05:29,商 品 S,NaN,浅田賢二
2,2019-05-11 19:42:07,商 品 a,NaN,南部慶二
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒
4,2019-04-22 03:09:35,商品a,NaN,平田鉄二


#### 入力ミスにより、欠損値や表記揺れが存在する → 「データの揺れ」

In [2]:
# 顧客データの読み込み
kokyaku_data = pd.read_excel('../input/bookfile2/kokyaku_daicho.xlsx')
kokyaku_data.head()

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018/01/04
1,岡田 敏也,おかだ としや,E市,okada_toshiya@example.com,42782
2,芳賀 希,はが のぞみ,A市,haga_nozomi@example.com,2018/01/07
3,荻野 愛,おぎの あい,F市,ogino_ai@example.com,42872
4,栗田 憲一,くりた けんいち,E市,kurita_kenichi@example.com,43127


#### データの揺れを解消し、整合性を撮るために、データの把握を行う。

> ## ノック12 : データの揺れを見る

In [3]:
uriage_data['item_name'].head()

0      商品A
1    商 品 S
2    商 品 a
3      商品Z
4      商品a
Name: item_name, dtype: object

* #### 空白が入っていたり、大文字と小文字の表記揺れがある。

In [4]:
uriage_data['item_price'].head()

0     100.0
1       NaN
2       NaN
3    2600.0
4       NaN
Name: item_price, dtype: float64

* #### 欠損値がある → 補完の仕方がデータ分析に影響する。

> ## ノック13 : データに揺れがあるまま集計する

In [5]:
# 商品の名前について、pivot_tableでまとめてどうなるか見てみる
uriage_data['purchase_date'] = pd.to_datetime(uriage_data['purchase_date'])
uriage_data['purchase_month'] = uriage_data['purchase_date'].dt.strftime('%Y%m')
res = uriage_data.pivot_table(index='purchase_month', columns='item_name', aggfunc='size', fill_value=0)
res


item_name,商品W,商 品 n,商品E,商品M,商品P,商品S,商品W,商品X,商 品O,商 品Q,...,商品k,商品l,商品o,商品p,商品r,商品s,商品t,商品v,商品x,商品y
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,0,1,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,1,1,0,0
201903,0,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201904,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
201905,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
201906,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
201907,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,2,0,0,0,0,0


 * #### 本来26個しかないはずの商品が、表記揺れのせいで99個あることになっている。
 * #### 商品Sや商品sなど本来同じ商品が別の商品として集計されている。

In [6]:
# 同様にitem_pricesに対して結果を見てみる
res = uriage_data.pivot_table(index='purchase_month', columns='item_name', values='item_price', aggfunc='sum', fill_value=0)
res

item_name,商品W,商 品 n,商品E,商品M,商品P,商品S,商品W,商品X,商 品O,商 品Q,...,商品k,商品l,商品o,商品p,商品r,商品s,商品t,商品v,商品x,商品y
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,0.0,1400.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1100.0,1200.0,1500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
201902,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2400.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1900.0,2000.0,2200.0,0.0,0.0
201903,0.0,0.0,500.0,1300.0,1600.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
201904,2300.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1700.0,...,0.0,0.0,0.0,0.0,0.0,1900.0,0.0,0.0,0.0,0.0
201905,0.0,0.0,0.0,0.0,0.0,1900.0,0.0,0.0,0.0,0.0,...,0.0,1200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2500.0
201906,0.0,0.0,0.0,0.0,0.0,0.0,2300.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1600.0,0.0,0.0,0.0,0.0,2400.0,0.0
201907,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1500.0,0.0,1800.0,0.0,0.0,0.0,0.0,0.0


* #### このまま集計を行なっても意味がないことがわかった。
  #### → データの揺れを補正するデータ加工を行う

## ノック14 : 商品名の揺れを補正する

In [7]:
# 補正後の結果が正しいかどうか判定するために、現状を把握する
# pd.uniqueで名前が異なる要素（ユニークな要素）の個数を取得
print("商品名の個数")
print(len(pd.unique(uriage_data['item_name'])))

商品名の個数
99


In [8]:
# スペースの削除を行い、半角全角を整える
uriage_data['item_name'] = uriage_data['item_name'].str.upper() # 小文字を大文字に変換（大文字はそのまま）
uriage_data['item_name'] = uriage_data['item_name'].str.replace(' ', "")  # 半角スペースを空白に置き換える
uriage_data['item_name'] = uriage_data['item_name'].str.replace('　', "") # 全角スペースを空白に置き換える
# item_nameに関して辞書順にソート
uriage_data = uriage_data.sort_values(by=['item_name'], ascending=True)

uriage_data

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,201906
1748,2019-05-19 20:22:22,商品A,100.0,松川綾女,201905
223,2019-06-25 08:13:20,商品A,100.0,板橋隆,201906
1742,2019-06-13 16:03:17,商品A,100.0,小平陽子,201906
1738,2019-02-10 00:28:43,商品A,100.0,松田浩正,201902
...,...,...,...,...,...
2880,2019-04-22 00:36:52,商品Y,NaN,田辺光洋,201904
2881,2019-04-30 14:21:09,商品Y,NaN,高原充則,201904
1525,2019-01-24 10:27:23,商品Y,2500.0,五十嵐春樹,201901
1361,2019-05-28 13:45:32,商品Y,2500.0,大崎ヒカル,201905


* ### 表をスクロールすると正しそうだが、「必ず」結果を数字で検証すること!!!

In [9]:
print("商品名 : ")
print(pd.unique(uriage_data['item_name']))
print("ユニークな商品の個数 : ")
print(len(pd.unique(uriage_data['item_name'])))

商品名 : 
['商品A' '商品B' '商品C' '商品D' '商品E' '商品F' '商品G' '商品H' '商品I' '商品J' '商品K' '商品L'
 '商品M' '商品N' '商品O' '商品P' '商品Q' '商品R' '商品S' '商品T' '商品U' '商品V' '商品W' '商品X'
 '商品Y' '商品Z']
ユニークな商品の個数 : 
26


* #### ユニークな商品の個数が26個になったので、商品名における表記揺れが解消したことがわかった。

## ノック15 : 金額欠損値の補完

In [10]:
# 欠損値が含まれているかどうかを列ごとに確認する
# axis=0で列ごと、axis=1で行ごとに確認できる（何も入力しないとaxis=0が自動で指定される）
uriage_data.isnull().any(axis=0)

purchase_date     False
item_name         False
item_price         True
customer_name     False
purchase_month    False
dtype: bool

* #### 金額に欠損値があることがわかる

In [11]:
# 欠損値の補完を行う
# 処理がややこしいので注意!
# 各要素に対して、item_price列の要素が欠損しているか取得
flg_is_null = uriage_data['item_price'].isnull()

# loc関数を使用して、条件に合致するデータの、商品名を取得
# ここではflg_is_null == Trueとなるものの商品名を取得
# .uniqueを用いて商品名の重複をなくし、欠損値の存在するリストを作成
# 欠損値リストにある商品名の中でループを回す
for trg in list(uriage_data.loc[flg_is_null, 'item_name'].unique()):
    # 「flg_is_nullがFalse（~は否定演算子）」 かつ 「商品名が現在のループの商品名と合致」
    # するものの商品価格を取得
    price = uriage_data.loc[(~flg_is_null) & (uriage_data['item_name'] == trg) , "item_price"].max()
    
    # 先ほど取得した商品価格を、欠損値の存在する行のitem_price列に代入
    uriage_data.loc[(flg_is_null) & (uriage_data['item_name'] == trg), 'item_price'] = price
    
uriage_data.head()

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,201906
1748,2019-05-19 20:22:22,商品A,100.0,松川綾女,201905
223,2019-06-25 08:13:20,商品A,100.0,板橋隆,201906
1742,2019-06-13 16:03:17,商品A,100.0,小平陽子,201906
1738,2019-02-10 00:28:43,商品A,100.0,松田浩正,201902


In [12]:
# 欠損値がなくなったかどうか確認
uriage_data.isnull().any()

purchase_date     False
item_name         False
item_price        False
customer_name     False
purchase_month    False
dtype: bool

* #### さらに、各商品の金額が正しく保管されたか確認する

In [13]:
for trg in list(uriage_data['item_name'].sort_values().unique()):
    print(trg + 'の最大値 : ' 
              + str(uriage_data.loc[uriage_data['item_name'] 
                                == trg]['item_price'].max())
              + 'の最小値 : ' 
              + str(uriage_data.loc[uriage_data['item_name'] 
                                == trg]['item_price'].min(skipna=False)))

商品Aの最大値 : 100.0の最小値 : 100.0
商品Bの最大値 : 200.0の最小値 : 200.0
商品Cの最大値 : 300.0の最小値 : 300.0
商品Dの最大値 : 400.0の最小値 : 400.0
商品Eの最大値 : 500.0の最小値 : 500.0
商品Fの最大値 : 600.0の最小値 : 600.0
商品Gの最大値 : 700.0の最小値 : 700.0
商品Hの最大値 : 800.0の最小値 : 800.0
商品Iの最大値 : 900.0の最小値 : 900.0
商品Jの最大値 : 1000.0の最小値 : 1000.0
商品Kの最大値 : 1100.0の最小値 : 1100.0
商品Lの最大値 : 1200.0の最小値 : 1200.0
商品Mの最大値 : 1300.0の最小値 : 1300.0
商品Nの最大値 : 1400.0の最小値 : 1400.0
商品Oの最大値 : 1500.0の最小値 : 1500.0
商品Pの最大値 : 1600.0の最小値 : 1600.0
商品Qの最大値 : 1700.0の最小値 : 1700.0
商品Rの最大値 : 1800.0の最小値 : 1800.0
商品Sの最大値 : 1900.0の最小値 : 1900.0
商品Tの最大値 : 2000.0の最小値 : 2000.0
商品Uの最大値 : 2100.0の最小値 : 2100.0
商品Vの最大値 : 2200.0の最小値 : 2200.0
商品Wの最大値 : 2300.0の最小値 : 2300.0
商品Xの最大値 : 2400.0の最小値 : 2400.0
商品Yの最大値 : 2500.0の最小値 : 2500.0
商品Zの最大値 : 2600.0の最小値 : 2600.0


* #### すべてのデータにおいて、最小値と最大値が一致している（つまり全商品の商品価格が同一）であることが確認できた。
* #### 加工前のデータで同一のコードを実行すると、商品Z以外は最小値がNaNになっていたので、正しく加工が行われていることが確認できる。

> ## ノック16 : 顧客名の揺れを補正する

In [14]:
# データの確認
kokyaku_data['顧客名'].head()

0     須賀ひとみ
1    岡田　 敏也
2      芳賀 希
3     荻野  愛
4     栗田 憲一
Name: 顧客名, dtype: object

In [15]:
uriage_data['customer_name'].head()

0       深井菜々美
1748     松川綾女
223       板橋隆
1742     小平陽子
1738     松田浩正
Name: customer_name, dtype: object

* 今回の場合、顧客名は売り上げデータと顧客台帳を結びつける唯一のキーなので、表記揺れの改善は必須
* 商品名の補正と同様に、スペースの除去を行う

In [16]:

kokyaku_data['顧客名'] = kokyaku_data['顧客名'].str.replace(' ', '') # 半角スペースを除去
kokyaku_data['顧客名'] = kokyaku_data['顧客名'].str.replace('　', '') # 全角スペースを除去
kokyaku_data['顧客名'].head()


0    須賀ひとみ
1     岡田敏也
2      芳賀希
3      荻野愛
4     栗田憲一
Name: 顧客名, dtype: object

* #### 今回は練習用データなので、表記揺れは空白のみで済んだ。
* #### 実際には誤変換など複雑な揺れが存在する
* #### 名前の誤変換の場合はプログラムでは修正できないので、地道にヒアリングを行なって改善する必要がある。
* #### 同姓同名のデータが存在する場合は登録日などを用いて区別する必要がある。

> ## ノック17 : 日付の揺れを補正する

In [17]:
kokyaku_data['登録日']

0      2018/01/04
1           42782
2      2018/01/07
3           42872
4           43127
          ...    
195    2017/06/20
196    2018/06/20
197    2017/04/29
198    2019/04/19
199    2019/04/23
Name: 登録日, Length: 200, dtype: object

* 一部データが数字になっている → yyyy年mm月dd日形式で書いたデータを読み込めていない

In [18]:
# 数字として読み込まれてしまった箇所を特定する

# 数字のままだと.str.isdigitメソッドが使えないので、
# .astype(str)で型をintからstrに変換している
flg_is_serial = kokyaku_data['登録日'].astype(str).str.isdigit()

print('数字で読み込まれてしまった日付の個数 : ' + str(flg_is_serial.sum()) + ' 個')

数字で読み込まれてしまった日付の個数 : 22 個


In [19]:
# 数字として読み込まれてしまった箇所を補正する

# to_timedeltaを用いて数値をdatetime型に変換
# なぜ-2なのかは、説明が長いので本を参照すること。
fromSerial = pd.to_timedelta(kokyaku_data.loc[flg_is_serial, '登録日']
                             .astype('float') - 2, unit='D') + pd.to_datetime('1900/1/1')
fromSerial

1     2017-02-16
3     2017-05-17
4     2018-01-27
21    2017-07-04
27    2017-06-15
47    2017-01-06
49    2017-07-13
53    2017-04-08
76    2018-03-29
80    2018-01-10
99    2017-05-30
114   2018-06-03
118   2018-01-29
122   2018-04-16
139   2017-05-25
143   2017-03-24
155   2017-01-19
172   2018-03-22
179   2017-01-08
183   2017-07-24
186   2018-07-13
192   2018-06-08
Name: 登録日, dtype: datetime64[ns]

* 表記揺れがあった26個のデータが正しく読み込まれた。

In [20]:
# 日付として取り込まれているデータを改めてdataframe型に変換する
fromString = pd.to_datetime(kokyaku_data.loc[~flg_is_serial, '登録日'])
fromString

0     2018-01-04
2     2018-01-07
5     2017-06-20
6     2018-06-11
7     2017-05-19
         ...    
195   2017-06-20
196   2018-06-20
197   2017-04-29
198   2019-04-19
199   2019-04-23
Name: 登録日, Length: 178, dtype: datetime64[ns]

In [21]:
# 二つのデータを結合して、データを更新する

# concatはインデックスを参照して対応する行に自動でデータをくっつけてくれるので
# 意図しない行に意図しない登録日がくっついてしまうことはない。
kokyaku_data['登録日'] = pd.concat([fromSerial, fromString])
kokyaku_data

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018-01-04
1,岡田敏也,おかだ としや,E市,okada_toshiya@example.com,2017-02-16
2,芳賀希,はが のぞみ,A市,haga_nozomi@example.com,2018-01-07
3,荻野愛,おぎの あい,F市,ogino_ai@example.com,2017-05-17
4,栗田憲一,くりた けんいち,E市,kurita_kenichi@example.com,2018-01-27
...,...,...,...,...,...
195,川上りえ,かわかみ りえ,G市,kawakami_rie@example.com,2017-06-20
196,小松季衣,こまつ としえ,E市,komatsu_toshie@example.com,2018-06-20
197,白鳥りえ,しらとり りえ,F市,shiratori_rie@example.com,2017-04-29
198,大西隆之介,おおにし りゅうのすけ,H市,oonishi_ryuunosuke@example.com,2019-04-19


In [22]:
# 登録日から登録月を算出し、集計してみる
kokyaku_data['登録年月'] = kokyaku_data['登録日'].dt.strftime('%Y%m')
# groupbyとcountで年月ごとにカウント。顧客名列を足し合わせた結果を表示
rsit = kokyaku_data.groupby('登録年月').count()['顧客名']
print(rsit)
print(len(kokyaku_data))

登録年月
201701    15
201702    11
201703    14
201704    15
201705    14
201706    13
201707    17
201801    13
201802    15
201803    17
201804     5
201805    19
201806    13
201807    17
201904     2
Name: 顧客名, dtype: int64
200


In [23]:
# 登録日列に数値データが残っていないか確認する
# 加工が正しければすべてdateframe型に変換されてるはず

flg_is_serial = kokyaku_data['登録日'].astype('str').str.isdigit()
flg_is_serial.sum()

0

* #### 最初は22件だった数値データが0件になったので、正しく加工ができていることが確認できた。

> ## ノック18 : 顧客名をキーに2つのデータを結合（ジョイン）する

In [24]:
# 共通するキー名がないのでそれぞれ指定する
# uriage_dataからは'customer_name'を、
# kokyaku_dataからは'顧客名'をキーに結合


join_data = pd.merge(uriage_data, kokyaku_data, left_on='customer_name',
                    right_on='顧客名', how='left')
# 今回の方法ではキーにした列が二つとも残る
# 情報の重複を防ぐために片方を消す
join_data = join_data.drop('customer_name', axis=1)

join_data

,purchase_date,item_name,item_price,purchase_month,顧客名,かな,地域,メールアドレス,登録日,登録年月
0,2019-06-13 18:02:34,商品A,100.0,201906,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26,201701
1,2019-05-19 20:22:22,商品A,100.0,201905,松川綾女,まつかわ あやめ,E市,matsukawa_ayame@example.com,2018-07-23,201807
2,2019-06-25 08:13:20,商品A,100.0,201906,板橋隆,いたばし たかし,B市,itabashi_takashi@example.com,2018-06-14,201806
3,2019-06-13 16:03:17,商品A,100.0,201906,小平陽子,こひら ようこ,B市,kohira_youko@example.com,2018-01-21,201801
4,2019-02-10 00:28:43,商品A,100.0,201902,松田浩正,まつだ ひろまさ,E市,matsuda_hiromasa@example.com,2018-01-11,201801
...,...,...,...,...,...,...,...,...,...,...
2994,2019-04-22 00:36:52,商品Y,2500.0,201904,田辺光洋,たなべ みつひろ,A市,tanabe_mitsuhiro@example.com,2018-07-04,201807
2995,2019-04-30 14:21:09,商品Y,2500.0,201904,高原充則,たかはら みつのり,H市,takahara_mitsunori@example.com,2018-04-21,201804
2996,2019-01-24 10:27:23,商品Y,2500.0,201901,五十嵐春樹,いがらし はるき,F市,igarashi_haruki@example.com,2017-01-06,201701
2997,2019-05-28 13:45:32,商品Y,2500.0,201905,大崎ヒカル,おおさき ひかる,E市,oosaki_hikaru@example.com,2017-02-22,201702


* #### このような分析に適したデータの加工のことを、「クレンジング」と呼ぶ。

> ## ノック19 : クレンジングしたデータをダンプする

* ### ダンプとは？ → データの出力のこと

ダンプする前に、データ列の並び順を整理しておく
（purchase_dateとpurchase_monthは近くにあった方がわかりやすいな...とか）

In [25]:
dump_data = join_data[['purchase_date', 'purchase_month', 'item_name', 'item_price', '顧客名', 'かな', '地域', 'メールアドレス', '登録日']]

dump_data

,purchase_date,purchase_month,item_name,item_price,顧客名,かな,地域,メールアドレス,登録日
0,2019-06-13 18:02:34,201906,商品A,100.0,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26
1,2019-05-19 20:22:22,201905,商品A,100.0,松川綾女,まつかわ あやめ,E市,matsukawa_ayame@example.com,2018-07-23
2,2019-06-25 08:13:20,201906,商品A,100.0,板橋隆,いたばし たかし,B市,itabashi_takashi@example.com,2018-06-14
3,2019-06-13 16:03:17,201906,商品A,100.0,小平陽子,こひら ようこ,B市,kohira_youko@example.com,2018-01-21
4,2019-02-10 00:28:43,201902,商品A,100.0,松田浩正,まつだ ひろまさ,E市,matsuda_hiromasa@example.com,2018-01-11
...,...,...,...,...,...,...,...,...,...
2994,2019-04-22 00:36:52,201904,商品Y,2500.0,田辺光洋,たなべ みつひろ,A市,tanabe_mitsuhiro@example.com,2018-07-04
2995,2019-04-30 14:21:09,201904,商品Y,2500.0,高原充則,たかはら みつのり,H市,takahara_mitsunori@example.com,2018-04-21
2996,2019-01-24 10:27:23,201901,商品Y,2500.0,五十嵐春樹,いがらし はるき,F市,igarashi_haruki@example.com,2017-01-06
2997,2019-05-28 13:45:32,201905,商品Y,2500.0,大崎ヒカル,おおさき ひかる,E市,oosaki_hikaru@example.com,2017-02-22


* #### せっかくクレンジングしたデータも出力しておかないとプログラムを閉じたときにもとに戻ってしまう。

In [26]:
# ダンプ（出力）する
dump_data.to_csv('dump_data.csv', index=False)

> ## ノック20 データを集計しよう

先ほど出力したデータを使って、集計を行う

In [27]:
# クレンジング後のデータの読み込み
import_data = pd.read_csv('dump_data.csv')
import_data

,purchase_date,purchase_month,item_name,item_price,顧客名,かな,地域,メールアドレス,登録日
0,2019-06-13 18:02:34,201906,商品A,100.0,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26
1,2019-05-19 20:22:22,201905,商品A,100.0,松川綾女,まつかわ あやめ,E市,matsukawa_ayame@example.com,2018-07-23
2,2019-06-25 08:13:20,201906,商品A,100.0,板橋隆,いたばし たかし,B市,itabashi_takashi@example.com,2018-06-14
3,2019-06-13 16:03:17,201906,商品A,100.0,小平陽子,こひら ようこ,B市,kohira_youko@example.com,2018-01-21
4,2019-02-10 00:28:43,201902,商品A,100.0,松田浩正,まつだ ひろまさ,E市,matsuda_hiromasa@example.com,2018-01-11
...,...,...,...,...,...,...,...,...,...
2994,2019-04-22 00:36:52,201904,商品Y,2500.0,田辺光洋,たなべ みつひろ,A市,tanabe_mitsuhiro@example.com,2018-07-04
2995,2019-04-30 14:21:09,201904,商品Y,2500.0,高原充則,たかはら みつのり,H市,takahara_mitsunori@example.com,2018-04-21
2996,2019-01-24 10:27:23,201901,商品Y,2500.0,五十嵐春樹,いがらし はるき,F市,igarashi_haruki@example.com,2017-01-06
2997,2019-05-28 13:45:32,201905,商品Y,2500.0,大崎ヒカル,おおさき ひかる,E市,oosaki_hikaru@example.com,2017-02-22


In [28]:
# インデックスを年月、列を商品名として集計
byItem = import_data.pivot_table(index='purchase_month', columns='item_name', 
                                 aggfunc='size', fill_value=0)
byItem

item_name,商品A,商品B,商品C,商品D,商品E,商品F,商品G,商品H,商品I,商品J,...,商品Q,商品R,商品S,商品T,商品U,商品V,商品W,商品X,商品Y,商品Z
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,18,13,19,17,18,15,11,16,18,17,...,17,21,20,17,7,22,13,14,10,0
201902,19,14,26,21,16,14,14,17,12,14,...,22,22,22,23,19,22,24,16,11,1
201903,17,21,20,17,9,27,14,18,12,16,...,23,16,20,12,23,18,16,21,16,0
201904,17,19,24,20,18,17,14,11,18,13,...,20,20,16,16,11,15,14,16,20,0
201905,24,14,16,14,19,18,23,15,16,11,...,13,22,18,16,16,9,21,16,20,0
201906,24,12,11,19,13,18,15,13,19,22,...,15,16,21,12,18,20,17,15,13,0
201907,20,20,17,17,12,17,19,19,19,23,...,15,19,23,21,13,28,16,18,12,0


正しく16商品の年月別の個数が算出できた。

In [29]:
byPrice = import_data.pivot_table(index='purchase_month', columns='item_name', 
                                  values='item_price', aggfunc='sum', fill_value=0)
byPrice

item_name,商品A,商品B,商品C,商品D,商品E,商品F,商品G,商品H,商品I,商品J,...,商品Q,商品R,商品S,商品T,商品U,商品V,商品W,商品X,商品Y,商品Z
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,1800.0,2600.0,5700.0,6800.0,9000.0,9000.0,7700.0,12800.0,16200.0,17000.0,...,28900.0,37800.0,38000.0,34000.0,14700.0,48400.0,29900.0,33600.0,25000.0,0.0
201902,1900.0,2800.0,7800.0,8400.0,8000.0,8400.0,9800.0,13600.0,10800.0,14000.0,...,37400.0,39600.0,41800.0,46000.0,39900.0,48400.0,55200.0,38400.0,27500.0,2600.0
201903,1700.0,4200.0,6000.0,6800.0,4500.0,16200.0,9800.0,14400.0,10800.0,16000.0,...,39100.0,28800.0,38000.0,24000.0,48300.0,39600.0,36800.0,50400.0,40000.0,0.0
201904,1700.0,3800.0,7200.0,8000.0,9000.0,10200.0,9800.0,8800.0,16200.0,13000.0,...,34000.0,36000.0,30400.0,32000.0,23100.0,33000.0,32200.0,38400.0,50000.0,0.0
201905,2400.0,2800.0,4800.0,5600.0,9500.0,10800.0,16100.0,12000.0,14400.0,11000.0,...,22100.0,39600.0,34200.0,32000.0,33600.0,19800.0,48300.0,38400.0,50000.0,0.0
201906,2400.0,2400.0,3300.0,7600.0,6500.0,10800.0,10500.0,10400.0,17100.0,22000.0,...,25500.0,28800.0,39900.0,24000.0,37800.0,44000.0,39100.0,36000.0,32500.0,0.0
201907,2000.0,4000.0,5100.0,6800.0,6000.0,10200.0,13300.0,15200.0,17100.0,23000.0,...,25500.0,34200.0,43700.0,42000.0,27300.0,61600.0,36800.0,43200.0,30000.0,0.0


In [30]:
# 年月ごとに顧客別の買った回数を集計
byCustomer = import_data.pivot_table(index='purchase_month', columns='顧客名', 
                                  aggfunc='size', fill_value=0)
byCustomer

顧客名,さだ千佳子,中仁晶,中田美智子,丸山光臣,久保田倫子,亀井一徳,五十嵐春樹,井上桃子,井口寛治,井川真悠子,...,香椎優一,高原充則,高梨結衣,高沢美咲,高田さんま,鳥居広司,鶴岡薫,麻生莉緒,黄川田博之,黒谷長利
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,3,1,4,2,2,0,5,3,3,1,...,0,1,1,1,5,2,0,2,2,5
201902,9,1,2,2,1,4,2,1,0,4,...,4,0,3,2,0,1,2,4,0,1
201903,1,2,1,6,1,4,3,3,2,2,...,3,1,6,2,4,2,4,2,2,1
201904,0,3,1,2,0,2,2,0,3,2,...,2,4,2,3,4,3,2,1,2,0
201905,3,2,5,2,4,1,2,1,3,3,...,1,1,1,0,2,2,3,4,4,1
201906,1,3,0,4,1,1,1,2,2,3,...,7,3,0,2,1,0,2,1,2,4
201907,3,0,3,2,5,3,5,2,5,5,...,2,4,4,2,0,2,4,3,4,1


In [31]:
# 年月別に地域別の販売回数を出す
byRegion = import_data.pivot_table(index='purchase_month', columns='地域',
                                  aggfunc='size', fill_value=0)
byRegion

地域,A市,B市,C市,D市,E市,F市,G市,H市
purchase_month,,,,,,,,
201901,59,55,72,34,49,57,49,42
201902,71,46,65,48,61,52,43,63
201903,64,52,57,43,52,59,51,59
201904,64,48,54,45,48,58,40,52
201905,57,52,68,48,59,65,35,43
201906,53,47,61,30,51,51,58,58
201907,76,53,61,42,54,64,47,54


In [32]:
# 集計期間で購入してないユーザーがいないかチェックする
away_data = pd.merge(uriage_data, kokyaku_data, left_on='customer_name',
                    right_on='顧客名', how='right')
# 購入日が空欄の顧客のデータを出す。
away_data[away_data['purchase_date'].isnull()][['顧客名', 'メールアドレス', '登録日']]

,顧客名,メールアドレス,登録日
2999,福井美希,fukui_miki1@example.com,2019-04-23
